### This code block is mostly taken from the tutorial

##### @ben There seems to be a test failing here on commit 63dca8bd5358515999086d8436bec2a6cf82e6fb
#### `FAILED tests/benchmark/test_matrix.py::TestBenchmark::test_matrix - ValueError: Root HP:0000118 is not in provided HPO!`
#### `================== 1 failed, 32 passed, 12 warnings in 14.15s ==================`

In [1]:
!cd .. && pytest

============================= test session starts ==============================
platform darwin -- Python 3.11.6, pytest-7.4.2, pluggy-1.3.0
rootdir: /Users/jtr4v/PythonProject/setsim
configfile: pytest.ini
plugins: anyio-3.7.1
collected 33 items                                                             

src/setsim/sim/phenomizer/_test__algo.py ......                          [ 18%]
src/setsim/sim/phenomizer/_test__io.py .                                 [ 21%]
tests/test_io.py ...                                                     [ 30%]
tests/benchmark/test_matrix.py ..F..                                     [ 45%]
tests/sim/test_ic.py .....                                               [ 60%]
tests/sim/test_setsim.py .............                                   [100%]

=================================== FAILURES ===================================
__________________________ TestBenchmark.test_matrix ___________________________
multiprocessing.pool.RemoteTraceback: 
"""
Trac

In [2]:
import os
import urllib.request
import subprocess

# URLs for resources
hp_json_url = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2024-04-26/hp.json'
hpoa_url = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2024-04-26/phenotype.hpoa'
phenpacket_store_github_url = 'https://github.com/monarch-initiative/phenopacket-store.git'
phenpacket_store_github_commit = 'c198254a3058478f82c9d4cfe46abe37c897db36'

# Paths for downloaded files and cloned repository
fpath_hpo = os.path.join(os.getcwd(), 'data/hp.json')
fpath_hpoa = os.path.join(os.getcwd(), 'data/phenotype.hpoa')
clone_dir = os.path.join(os.getcwd(), 'data/phenopacket-store')
fpath_phenopackets = os.path.join(clone_dir, 'notebooks')

# Download hp.json if necessary
if not os.path.exists(fpath_hpo):
    os.makedirs(os.path.dirname(fpath_hpo), exist_ok=True)
    urllib.request.urlretrieve(hp_json_url, fpath_hpo)

# Download phenotype.hpoa if necessary
if not os.path.exists(fpath_hpoa):
    os.makedirs(os.path.dirname(fpath_hpoa), exist_ok=True)
    urllib.request.urlretrieve(hpoa_url, fpath_hpoa)

# Clone the phenopacket-store repository if necessary
if not os.path.exists(clone_dir):
    subprocess.run(['git', 'clone', phenpacket_store_github_url, clone_dir])

# Check out the specific commit
subprocess.run(['git', 'checkout', phenpacket_store_github_commit], cwd=clone_dir)

HEAD is now at c198254a BCKDHB


CompletedProcess(args=['git', 'checkout', 'c198254a3058478f82c9d4cfe46abe37c897db36'], returncode=0)

In [5]:
!cd .. && python3 -m pip install -e .[test]

Obtaining file:///Users/jtr4v/PythonProject/setsim
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build editable did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      No `packages` or `py_modules` configuration, performing automatic discovery.
      `src-layout` detected -- analysing ./src
      discovered packages -- ['setsim', 'sumsim', 'setsim._run_scripts', 'setsim.tests', 'setsim.model', 'setsim.sim', 'setsim.matrix', 'setsim.tests.data', 'setsim.sim.phenomizer', 'setsim.sim.phenomizer.test_data', 'sumsim.model', 'sumsim.sim', 'sumsim.sim.phenomizer']
      discovered py_modules -- []
      Traceback (most recent call last):
        File "/Users/jtr4v/PythonProject/setsim/venv/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
    

In [6]:
# Add the src directory to the Python path
src_path = os.path.abspath(os.path.join(os.getcwd(), '../src'))
if src_path not in sys.path:
    sys.path.append(src_path)

import sys
import os
from hpotk.ontology.load.obographs import load_minimal_ontology
from hpotk.ontology import MinimalOntology

from setsim.io import read_folder, read_hpoa
import warnings

# Define file paths
fpath_hpo = 'path_to_hpo.json'
fpath_phenopackets = 'path_to_phenopackets_folder'
fpath_hpoa = 'path_to_hpoa'

# Import hpo.json file
hpo: MinimalOntology = load_minimal_ontology(fpath_hpo)

# Import samples. Recursive import finds phenopackets in subfolders.
# read_folder and read_hpoa will generate warnings from phenopackets with redundant terms (ancestors of other included terms)
# and terms that cannot be read. We will filter warnings for this step to silence these.
warnings.filterwarnings('ignore')
samples = read_folder(fpath_phenopackets, hpo, recursive=True)

# Samples with no features should always have a similarity of 0. We will remove them to avoid problems.
samples = [sample for sample in samples if len(sample.phenotypic_features) > 0]

# Import diseases in a similar format to samples (individuals).
diseases = read_hpoa(fpath_hpoa, hpo)

# Turn warnings back to default
warnings.filterwarnings('default')

from setsim.sim import IcCalculator, IcTransformer

# The IcCalculator class object will be used to create a dictionary with term ids and ic values.
# IC values are calculated as the base 10 log of the number of diseases divided by the number of diseases annotated with that term.
# Diseases are annotated with a term when the disease is annotated with that term or any of its descendants.
calc = IcCalculator(hpo, multiprocess=True, progress_bar=True)
ic_dict = calc.calculate_ic_from_diseases(diseases) # alternatively "calculate_ic_from_samples" can be used to calculate ic using feature 
                                                    # prevalence in samples.

import math
import random
from statistics import mean
from setsim.model import Sample

common_terms = [term for term in ic_dict.keys() if ic_dict[term] <= math.log(10)]
noisy_samples = []
for sample in samples:
    noisy_features = list(sample.phenotypic_features) + list(random.sample(common_terms, 20))
    noisy_label = sample.label + "noisy"
    noisy_samples.append(
        Sample(label=noisy_label, phenotypic_features=noisy_features, disease_identifier=sample.disease_identifier,
               hpo=hpo))

print(f"There are {len(common_terms)} common terms with IC equal to or less than ln(10) ({math.log(10)}).")
print(f'The samples have an average of {mean([len(sample.phenotypic_features) for sample in samples])} terms after removing ancestors and duplicates.')
print(f'The noisy samples (with 20 random common terms added) have an average of {mean([len(sample.phenotypic_features) for sample in noisy_samples])} terms after removing ancestors and duplicates.')

ModuleNotFoundError: No module named 'google'

### Get a list of diseases from the diagnoses of the patients in phenopacket-store

In [2]:
sample_disease_list = [sample.disease_identifier.identifier.value for sample in samples]
disease_counts = dict()
for disease in sample_disease_list:
  disease_counts[disease] = disease_counts.get(disease, 0) + 1

### Identify diseases diagnosed to at least 30 individuals from phenopacket-store

In [3]:
disease30p_list = []
for disease, n in disease_counts.items():
    if n >= 30:
        disease30p_list.append(disease)

In [4]:
disease30p = [disease for disease in diseases if disease.identifier.value in disease30p_list]

### Create IC Dictionary using only the diseases without 30 people diagnosed
(Consider skipping this step.)

In [5]:
for disease in disease30p:
    diseases.remove(disease)

In [7]:
# The IcCalculator class object will be used to create a dictionary with term ids and ic values.
# IC values are calculated as the base 10 log of the number of diseases divided by the number of diseases annotated with that term.
# Diseases are annotated with a term when the disease is annotated with that term or any of its descendants.
calc = IcCalculator(hpo, multiprocess=True, progress_bar=True)
ic_dict = calc.calculate_ic_from_diseases(diseases) # alternatively "calculate_ic_from_samples" can be used to calculate ic using feature 
                                                    # prevalence in samples.

# The IcTransformer calculates the  conditional information for each term given its parents.
# We only need to supply diseases if we are using the method, phrank, which requires additional information to approximate conditional information.
transformer = IcTransformer(hpo, samples=diseases)
delta_ic_dict = transformer.transform(ic_dict)
bayes_ic_dict = transformer.transform(ic_dict, strategy="bayesian") # Used for phrank

### Identify individuals with one of the chosen diseases

In [8]:
i = 0
samples_30p = []
for d_name in sample_disease_list:
    if d_name in disease30p_list:
        samples_30p.append(samples[i])
    i = i + 1

In [9]:
len(set([d.disease_identifier.identifier.value for d in samples_30p]))

27

In [11]:
len(disease30p_list)

27

## Run Clustering
This should only need to be run twice. Once for normal patients (the subset with the 30p diseases) and once for the same patients with common terms added (noisy patients).

In [13]:
from setsim.matrix import SimilarityMatrix
# Need to change this to include all methods
methods = ['phenomizer', 'count', 'simici'] #['phenomizer', 'count', 'simici', 'phrank', 'jaccard', 'simgic', 'simgci']

# Run Sim Matrix
sim_matrix = SimilarityMatrix(hpo=hpo,
                              chunksize=13, # Chunksize refers to the number of diseases per "chunk". 
                                           # Smaller chunksizes are often optimal because diseases with more features take longer to run.
                              delta_ic_dict=delta_ic_dict, # Used for simici and simgci
                              ic_dict=ic_dict, # Used for phenomizer and simgic
                              bayes_ic_dict=bayes_ic_dict, # Used for phrank
                              n_iter_distribution=0, # This is 0 because we aren't calculating p-values
                              num_features_distribution = 1,
                              num_cpus=15,
                              patients=samples_30p,
                              similarity_methods=methods,
                              multiprocess=True
                              )
results = sim_matrix.compute_person2person_similarities(samples_30p)

There are 16 CPUs available for multiprocessing. Using 15 CPUs.


Diseases: 100%|█████████████████████████████| 2090/2090 [21:30<00:00,  1.62it/s]


In [14]:
results

,disease_id,num_features,Individual 28_phenomizer_sim,Individual 28_count_sim,Individual 28_simici_sim,Individual 31_phenomizer_sim,Individual 31_count_sim,Individual 31_simici_sim,Individual 76_phenomizer_sim,Individual 76_count_sim,...,34 _simici_sim,26 (twin of patient 25)_phenomizer_sim,26 (twin of patient 25)_count_sim,26 (twin of patient 25)_simici_sim,24 (mother of patients 25 and 26)_phenomizer_sim,24 (mother of patients 25 and 26)_count_sim,24 (mother of patients 25 and 26)_simici_sim,"33 (also see Urreizti et al AJMG 2016, patient P6, for phenotypic description)_phenomizer_sim","33 (also see Urreizti et al AJMG 2016, patient P6, for phenotypic description)_count_sim","33 (also see Urreizti et al AJMG 2016, patient P6, for phenotypic description)_simici_sim"
43,OMIM:617140,21,0.472742,14.0,4.844663,0.956359,30.0,12.278309,0.940891,30.0,...,7.557433,0.725357,19.0,7.132501,0.954753,30.0,10.643218,1.538135,52.0,20.605697
6,OMIM:617140,13,0.765011,13.0,4.912108,1.371874,24.0,11.559063,1.151293,17.0,...,7.844645,0.815254,15.0,5.495117,1.047183,17.0,6.738498,1.449316,26.0,11.168889
38,OMIM:617140,10,1.347760,16.0,9.805472,1.468438,18.0,11.012247,1.211741,17.0,...,8.691087,0.797336,10.0,3.589712,1.155550,18.0,6.295353,1.437442,21.0,8.316940
45,OMIM:617140,11,1.071033,22.0,8.955370,1.192341,27.0,9.459466,1.311160,31.0,...,6.951520,1.076644,29.0,9.853715,1.038379,28.0,7.412090,1.334125,40.0,12.024259
7,OMIM:617140,14,1.065502,22.0,11.711275,1.123776,23.0,11.284843,0.937355,25.0,...,7.454588,0.752311,23.0,7.897811,1.087707,30.0,11.085999,1.596474,43.0,17.411394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,OMIM:130050,3,0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.760063,2.0,...,1.636312,0.760063,2.0,1.140094,1.090875,3.0,1.636312,0.000000,1.0,0.000000
13,OMIM:130050,6,0.124606,3.0,0.747637,0.290466,7.0,1.742797,0.670498,8.0,...,5.325479,0.605408,8.0,2.492356,1.446542,17.0,7.518631,1.202832,15.0,7.216992
17,OMIM:130050,6,0.524558,7.0,3.147349,0.437142,14.0,4.767808,1.062257,17.0,...,2.164182,0.605408,8.0,2.492356,1.094949,20.0,7.483677,0.549511,18.0,5.847365
10,OMIM:130050,2,0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.000000,1.0,...,4.797609,0.000000,1.0,0.000000,1.629540,7.0,3.734767,0.000000,1.0,0.000000


# Next Steps
(The rest of the code is an old model.)
## For one iteration:
    Select 5 diseases (from those with 30 patients)
    From results dataframe, keep rows and columns of individuals with 1 of five diseases only
    For each method:
        create similarity matrix from selected results
        Run kmeans with 5 clusters
        Get fraction of individuals that are correctly clustered

## Overall
    Repeat for 100-1000 unique combinations of diseases
    Calculate p-values
    Repeat for noisy patients
        Don't need to recalculate ic_dict 
        Start by rerunning SimilarityMatrix

In [ ]:
similarity_matrices

In [ ]:
# create list of a thousand lists of 5 diseases

# iterate through lists
for disease_subset in diseases_sets:
    

In [ ]:
def roc_score(pred, standard, cluster_diseases_list):
    disease_list = list(set(standard))
    tpr_fpr = {}
    total_true_pos = 0
    for d in cluster_diseases_list:
        cluster_assignment = [pred[i] for i in range(len(standard)) if standard[i] == d]
        assignment_count = Counter(cluster_assignment)
        disease_cluster = max(assignment_count, key=assignment_count.get)
        true_pos = assignment_count[disease_cluster]        
        total_true_pos = total_true_pos + true_pos
        false_pos = sum(True if pred[i] == disease_cluster else False for i in range(len(standard)) if standard[i] != d)
        #true_neg = len(pred) - false_pos - len(cluster_assignment)
        tpr_fpr[d] = (true_pos/len(cluster_assignment), false_pos/(len(pred) - len(cluster_assignment)))
    tpr_fpr["total_true_pos"] = total_true_pos/len(pred)
    return tpr_fpr

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter

scores = {}
for method in methods:
    cols = [col for col in similarity_matrices.columns if f"_{method}_sim" in col]
    kmeans = KMeans(n_clusters=5, random_state=42, n_init="auto").fit(similarity_matrices[cols])
    scores[method] = roc_score(kmeans.fit_predict(similarity_matrices[cols])[:-20], similarity_matrices["disease_id"].iloc[:-20], cluster_diseases_list)

In [ ]:
import umap
simgci_cols = [col for col in similarity_matrices.columns if "simgci" in col]
simgci_df = similarity_matrices[simgci_cols]
pc_sim_matrix = umap.UMAP(random_state=42).fit_transform(simgci_df)
similarity_matrices["UMAP 1"] = pc_sim_matrix[:,0]
similarity_matrices["UMAP 2"] = pc_sim_matrix[:,1]


In [ ]:
cluster_disease_values = [dis.identifier.value for dis in cluster_diseases]
similarity_matrices["Disease"] = [dis if dis in cluster_disease_values else "control" for dis in similarity_matrices["disease_id"]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11.7,8.27)})
g = sns.scatterplot(data=similarity_matrices, x="UMAP 1", y="UMAP 2",hue="Disease", palette="bright", alpha = 0.5)
sns.move_legend(g, "lower left")
#g.set_ylim(0, 15)
#g.set_xlim(0, 15)
plt.savefig("simgci_cluster.png")

## Repeat for noisy patients

In [ ]:
clustering_samples = [sample for sample in noisy_samples if sample.disease_identifier.identifier.value in cluster_diseases_list]
clustering_samples = clustering_samples + [noisy_samples[i] for i in extra_sample_index]

In [ ]:
b_mark = Benchmark(hpo=hp,
                   chunksize=4,
                   delta_ic_dict=delta_ic_dict, 
                   bayes_ic_dict = bayes_ic_dict,
                   ic_dict=ic_dict,
                   mica_dict=mica_dict,
                   n_iter_distribution=0,
                   num_cpus=14,
                   num_features_distribution=30,
                   patients=clustering_samples,
                   similarity_methods = methods,
                   multiprocess=True
                  )
similarity_matrices_noisy = b_mark.compute_person2person_similarities(clustering_samples)

In [ ]:
scores_noisy = {}
for method in methods:
    cols = [col for col in similarity_matrices_noisy.columns if f"_{method}_sim" in col]
    kmeans = KMeans(n_clusters=5, random_state=42, n_init="auto").fit(similarity_matrices_noisy[cols])
    scores_noisy[method] = roc_score(kmeans.fit_predict(similarity_matrices_noisy[cols])[:-20], similarity_matrices_noisy["disease_id"].iloc[:-20], cluster_diseases_list)

In [ ]:
df_arrange_results = pd.DataFrame(scores).transpose()
df_arrange_results['samples'] = 'normal'
df_arrange_results = df_arrange_results[['total_true_pos','samples']]

df_ar_noisy = pd.DataFrame(scores_noisy).transpose()
df_ar_noisy['samples'] = 'noisy'
df_ar_noisy = df_ar_noisy[['total_true_pos','samples']]

In [ ]:
graphable_results = pd.concat([df_arrange_results, df_ar_noisy])

In [ ]:
original_names = ["phenomizer", "sumsim", "phrank", "jaccard", "simgic", "simcic", "count"]
new_names = ["Phenomizer", "SimICI", "Phrank", "Jaccard", "SimGIC", "SimGCI", "Count"]
name_conversion = {k: v for k, v in zip(original_names, new_names)}
graphable_results.index = [name_conversion[idx] for idx in graphable_results.index]
graphable_results["algorithm"] = graphable_results.index.tolist()

In [ ]:
graphable_results

In [ ]:
import seaborn as sns
g = sns.catplot(data=graphable_results, 
                hue = "algorithm", 
                y='total_true_pos', 
                x='samples', 
                order = ['normal','noisy'], 
                kind="point", 
                hue_order=["Phenomizer", "Count", "SimICI", "Phrank", "Jaccard", "SimGIC", "SimGCI"])
g.set(ylabel='Fraction Correctly Clustered')

In [ ]:
g.savefig("clustering.png")